In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [55]:
df=pd.read_csv(r"C:\Users\Nidhi\Desktop\myntra\dataverse_files\fashion data on Instagram.csv")

In [56]:
df.head()

,UserId,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,...,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,...,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181
1,1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,...,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,...,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,...,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,...,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [57]:
# converting unix dates to yyyy-mm-dd
df['Date'] = pd.to_datetime(df['CreationTime'], unit='s').dt.strftime('%Y-%m-%d')

In [58]:
df=df.dropna()

In [59]:
#convert encoded bytes to readable
def decode_unicode(text):
    return bytes(text, 'utf-8').decode('unicode_escape')

df['Hashtags'] = df['Hashtags'].str.split(', ').apply(lambda tags: [decode_unicode(tag) for tag in tags])

print(df['Hashtags'])

0        [beautiful, summer, fashion, love, cute, food,...
1        [teen, model, brunette, selfie, hollister, sna...
2        [alexandani, cute, llbean, beanboots, anthropo...
3        [alexandani, cute, llbean, beanboots, anthropo...
4        [abercrombieandfitch, shopping, love, holliste...
                               ...                        
24747    [เสื้อเท่ๆ, เสื้อใส่สบาย, เสื้อไปทะเล, เสื้อสว...
24748    [siambrandname, topshopthailand, somethingboud...
24749    [siambrandname, coach, peony, zara, รับตัวแทน,...
24750    [siambrandname, coach, peony, zara, รับตัวแทน,...
24751    [siambrandname, topshopthailand, somethingboud...
Name: Hashtags, Length: 24589, dtype: object


In [60]:
#get useful columns
df_required = df[['Date', 'Hashtags']]

In [61]:
# separate hashtags
df_exploded = df_required.explode('Hashtags')

In [62]:
# get date in datetime format
df_required['Date'] = pd.to_datetime(df_required['Date'])

C:\Users\Nidhi\AppData\Local\Temp\ipykernel_22944\2866156661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_required['Date'] = pd.to_datetime(df_required['Date'])


In [63]:
# separate hashtags
df_exploded = df_required.explode('Hashtags')

In [64]:
# get date in datetime format
df_required['Date'] = pd.to_datetime(df_required['Date'])

C:\Users\Nidhi\AppData\Local\Temp\ipykernel_22944\2866156661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_required['Date'] = pd.to_datetime(df_required['Date'])


In [67]:
df_exploded=df_exploded.set_index(['Date'])

In [69]:
# groupby weekly trends
hashtag_trends = df_exploded.groupby('Hashtags').resample('W-Mon').size().reset_index(name='Count')

In [71]:
hashtag_trends=hashtag_trends.set_index(['Date'])

In [75]:
hashtag_trends_pivot = hashtag_trends.pivot(columns='Hashtags', values='Count').fillna(0)

In [91]:
# filter to get dates after 2015
filtered_hashtag_trends_pivot = hashtag_trends_pivot[hashtag_trends_pivot.index >= "2015-01-01"]

In [92]:
df_long_term = filtered_hashtag_trends_pivot.copy()

In [94]:
hashtag_slopes = {}

In [95]:

# Fit linear regression and calculate slopes
for hashtag in df_long_term.columns:
    y = df_long_term[hashtag].values.reshape(-1, 1)
    X = np.arange(len(df_long_term)).reshape(-1, 1)  # Use indices as X for simplicity
    model = LinearRegression()
    model.fit(X, y)
    slope = model.coef_[0][0]  # Slope of the linear regression line
    hashtag_slopes[hashtag] = slope

In [96]:
sorted_hashtag_slopes = sorted(hashtag_slopes.items(), key=lambda x: x[1], reverse=True)

In [97]:
# Display top trending hashtags based on slopes
print("Top Trending Hashtags (Long-term):")
for tag, slope in sorted_hashtag_slopes[:10]:  # Displaying top 10 for brevity
    print(f"{tag}: Slope={slope:.2f}")

Top Trending Hashtags (Long-term):
fashion: Slope=8.61
chanel: Slope=5.01
ootd: Slope=4.60
style: Slope=4.25
prada: Slope=3.78
dior: Slope=3.62
gucci: Slope=3.31
hermes: Slope=3.26
zara: Slope=2.98
louisvuitton: Slope=2.66


using arima forecasting the hashtags

In [111]:
# Filter the data for the top trending hashtags
top_hashtags = ['fashion', 'chanel', 'ootd', 'style', 'prada', 'dior', 'gucci', 'hermes', 'zara', 'louisvuitton']
df_filtered = df_long_term[top_hashtags]

In [113]:
def forecast_hashtag(df, hashtag, forecast_steps=13, order=(1, 1, 1)):
    series = df[hashtag]
    data = series.values.astype(float)
    
    # Fit ARIMA model
    model = ARIMA(data, order=order)
    model_fit = model.fit()
    
    # Forecast
    forecast = model_fit.forecast(steps=forecast_steps)
    
    return forecast


In [114]:
forecasts = {}

for hashtag in top_hashtags:
    forecasts[hashtag] = forecast_hashtag(df_filtered, hashtag)

C:\Users\Nidhi\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Nidhi\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [124]:
# Calculate mean forecast for each hashtag
mean_forecasts = {hashtag: sum(values) / len(values) for hashtag, values in forecasts.items()}

# Sort mean forecasts by value (descending)
sorted_mean_forecasts = sorted(mean_forecasts.items(), key=lambda x: x[1], reverse=True)

# Print the hashtag with the highest mean forecast
highest_hashtag, highest_mean_forecast = sorted_mean_forecasts[0]
print(f"The hashtag with the highest mean forecast is '{highest_hashtag}' with a mean forecast of {highest_mean_forecast:.2f} over next 13 weeks")

The hashtag with the highest mean forecast is 'zara' with a mean forecast of 15896.43 over next 13 weeks
